In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
from spacy.lang.en import stop_words
import nltk
from nltk.stem import WordNetLemmatizer
from collections import Counter
from sklearn.model_selection import train_test_split

In [10]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sayan\AppData\Roaming\nltk_data...


True

In [11]:
emails = pd.read_csv(r"D:\code\AppliedML_assign_02\Data\emails.csv")
emails.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


Pre-Processing the Dataset:

In [12]:
emails.text = emails.text.astype(str).map(lambda x:x[9:])
emails.text = emails.text.str.replace('\d', '')
emails.text = emails.text.astype(str).map(lambda x:re.sub(r'http\S+', '', x))
emails.text = emails.text.astype(str).map(lambda x:re.sub(r"\S*@\S*\s?",'', x))
emails.text = emails.text.astype(str).map(lambda x:re.sub('\W+',' ', x))
emails.text = emails.text.map(lambda x:x.lower())
emails.text = emails.text.map(lambda x:x.translate(str.maketrans('', '', string.punctuation)))

In [13]:
stop_words = stop_words.STOP_WORDS
emails.text = emails.text.map(lambda x: " ".join(_ for _ in x.split() if _ not in stop_words))

In [14]:
# Using lemmatization for getting the base words
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()
emails.text = emails.text.map(lambda x: " ".join(lemmatizer.lemmatize(_) for _ in x.split()))

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\sayan\AppData\Roaming\nltk_data...


In [15]:
combined_text = " ".join(emails.text)
vocab = Counter(combined_text.split())
new_sw = list({key: value for key, value in vocab.items() if len(key) == 1}.keys())
emails.text = emails.text.map(lambda x: " ".join(_ for _ in x.split() if _ not in new_sw))
emails.head()

,text,spam
0,naturally irresistible corporate identity lt h...,1
1,stock trading gunslinger fanny merrill muzo co...,1
2,unbelievable new home easy im wanting homeowne...,1
3,color printing special request additional info...,1
4,money software cd software compatibility ain g...,1


In [16]:
X, y = emails.text, emails.spam
train_X, test_X, train_y, test_y = train_test_split(X, y, shuffle=True, train_size=0.8, random_state=1)
train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, shuffle=True, train_size=0.8, random_state=1)

In [17]:
print("Train set - number of 0s:", sum(train_y == 0), "& number of 1s:", sum(train_y == 1))
print("Validation set - number of 0s:", sum(val_y == 0), "& number of 1s:", sum(val_y== 1))
print("Test set - number of 0s:", sum(test_y== 0), "& number of 1s:", sum(test_y== 1))

Train set - number of 0s: 2791 & number of 1s: 874
Validation set - number of 0s: 693 & number of 1s: 224
Test set - number of 0s: 876 & number of 1s: 270


Datasets to csv:

In [18]:
train_data = pd.concat([train_X, train_y], axis=1)
train_data.to_csv(r'D:\code\AppliedML_assign_02\Data\train.csv', index=False)

test_data = pd.concat([test_X, test_y], axis=1)
test_data.to_csv(r'D:\code\AppliedML_assign_02\Data\test.csv', index=False)

val_data = pd.concat([val_X, val_y], axis=1)
val_data.to_csv(r'D:\code\AppliedML_assign_02\Data\validation.csv', index=False)

In [1]:
!git init

Initialized empty Git repository in D:/code/AppliedML_assign_02/.git/


In [2]:
!dvc init

Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In [6]:
! dvc add Data
! git add Data.dvc
! git commit -m "Tracking Data"


To track the changes with git, run:

	git add Data.dvc

To enable auto staging, run:

	dvc config core.autostage true


⠋ Checking graph



[master (root-commit) cff330a] Tracking Data
 4 files changed, 12 insertions(+)
 create mode 100644 .dvc/.gitignore
 create mode 100644 .dvc/config
 create mode 100644 .dvcignore
 create mode 100644 Data.dvc


In [21]:
! dvc remote add -df storage gdrive://1RyERR8UR6PufKeM7kh0-oxzsknqdFAwp
! dvc push

Setting 'storage' as a default remote.
Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=710796635688-iivsgbgsb6uv1fap6635dhvuei09o66c.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.appdata&access_type=offline&response_type=code&approval_prompt=force

Authentication successful.
2 files pushed


C:\Users\sayan\AppData\Local\Programs\Python\Python311\Lib\site-packages\oauth2client\_helpers.py:255: UserWarning: Cannot access C:\Users\sayan\AppData\Local\pydrive2fs\Cache\710796635688-iivsgbgsb6uv1fap6635dhvuei09o66c.apps.googleusercontent.com\default.json: No such file or directory
  warnings.warn(_MISSING_FILE_MESSAGE.format(filename))


In [22]:
X, y = emails.text, emails.spam
train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.8, random_state=100)
train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, train_size=0.8, random_state=100)

In [23]:
print("Train set - number of 0s:", sum(train_y == 0), "& number of 1s:", sum(train_y == 1))
print("Validation set - number of 0s:", sum(val_y == 0), "& number of 1s:", sum(val_y== 1))
print("Test set - number of 0s:", sum(test_y== 0), "& number of 1s:", sum(test_y== 1))

Train set - number of 0s: 2818 & number of 1s: 847
Validation set - number of 0s: 697 & number of 1s: 220
Test set - number of 0s: 845 & number of 1s: 301


In [24]:
! dvc add Data
! git add Data.dvc
! git commit -m "Seed Changes"
! dvc push


To track the changes with git, run:

	git add Data.dvc

To enable auto staging, run:

	dvc config core.autostage true


⠋ Checking graph



[master 4de9dc4] Seed Changes
 1 file changed, 3 insertions(+), 3 deletions(-)
4 files pushed


In [25]:
! git log --oneline

4de9dc4 Seed Changes
cff330a Tracking Data


In [26]:
! git fetch
! git checkout HEAD Data.dvc      
! dvc checkout     

Updated 0 paths from fc54450


In [27]:
X, y = emails.text, emails.spam
train_X, test_X, train_y, test_y = train_test_split(X, y, shuffle=True, train_size=0.8)
train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, shuffle=True, train_size=0.8)

In [28]:
print("Train set - number of 0s:", sum(train_y == 0), "& number of 1s:", sum(train_y == 1))
print("Validation set - number of 0s:", sum(val_y == 0), "& number of 1s:", sum(val_y== 1))
print("Test set - number of 0s:", sum(test_y== 0), "& number of 1s:", sum(test_y== 1))

Train set - number of 0s: 2810 & number of 1s: 855
Validation set - number of 0s: 706 & number of 1s: 211
Test set - number of 0s: 844 & number of 1s: 302


In [29]:
! git checkout HEAD Data.dvc      
! dvc checkout    

Updated 0 paths from fc54450


In [30]:
X, y = emails.text, emails.spam
train_X, test_X, train_y, test_y = train_test_split(X, y, shuffle=True, train_size=0.8)
train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, shuffle=True, train_size=0.8)

In [31]:
print("Train set - number of 0s:", sum(train_y == 0), "& number of 1s:", sum(train_y == 1))
print("Validation set - number of 0s:", sum(val_y == 0), "& number of 1s:", sum(val_y== 1))
print("Test set - number of 0s:", sum(test_y== 0), "& number of 1s:", sum(test_y== 1))

Train set - number of 0s: 2773 & number of 1s: 892
Validation set - number of 0s: 698 & number of 1s: 219
Test set - number of 0s: 889 & number of 1s: 257
